In [1]:
import math

In [2]:
SECONDS_PER_YEAR = 365 * 24 * 60 * 60

In [3]:
def apr_to_apy(apr: float, compounds: int) -> float:
    if compounds <= 0:
        return apr
    periodic = apr / compounds
    return (1 + periodic) ** compounds - 1

In [ ]:
# reward_rate : same value as given by `ProtocolStaking:rewardRate()`
# num_tokens_per_pool: same values as given by `ProtocolStaking:balanceOf(address(OperatorStaking))` for every eligible OperatorStaking
# fees_per_pool: same values as given by `OperatorRewarder:feeBasisPoints()` for every corresponding OperatorRewarder
def compute_native_APR(reward_rate: int, num_tokens_per_pool: list[int], fees_per_pool: list[int]) -> list[float]:
    """
    :param reward_rate: in amount of tokens per second according to the EVM, i.e reward_rate of 1e18 <=> 1 ZAMA token per second
    :param num_tokens_per_pool: list of amount of tokens deposited in each OperatorStaking pool
    :param fees_per_pool: list of fees, one per OperatorStaking pool, in basis points, i.e fee of 10000 <=> 100% fees 
    
    :return: a float corresponding to the PERCENTAGE of _native_ APR i.e not considering direct token donations to OperatorStaking.
    It assumes reward_rate, num_tokens_per_pool and fees_per_pool are constants, to get an instantaneous APR value.
    """
    assert len(num_tokens_per_pool)==len(fees_per_pool), "`num_tokens_per_pool` and `fees_per_pool` length mismatch"
    assert all(0 <= fee <= 10000 for fee in fees_per_pool), "fees must be in [0, 10000]"
    assert all(0 <= num_token for num_token in num_tokens_per_pool), "number of tokens must be non-negative"
    assert reward_rate >=0, "`reward_rate` must be non-negative"

    weights = [int(math.sqrt(x)) for x in num_tokens_per_pool]
    total_weight = sum(weights)
    fee_factors = [1-fee/10000  for fee in fees_per_pool]
    reward_rate_per_sec_per_pool = [reward_rate * (pool_weight / total_weight) for pool_weight in weights]
    pool_aprs = []
    for i in range(len(fee_factors)):
        net_reward_per_sec_per_pool = reward_rate_per_sec_per_pool[i] * fee_factors[i]
        pool_aprs.append((net_reward_per_sec_per_pool / num_tokens_per_pool[i]) * SECONDS_PER_YEAR * 100)
    
    return pool_aprs

In [ ]:
def compute_native_APY(
    reward_rate: int,
    num_tokens_per_pool: list[int],
    fees_per_pool: list[int],
    compounds: int,
) -> list[float]:
    """
    Same inputs as compute_native_APR plus `compounds` per year.
    Returns APY as PERCENT (e.g. 5.1 for 5.1%).
    """
    aprs_percent = compute_native_APR(reward_rate, num_tokens_per_pool, fees_per_pool)
    apys_percent = [
        apr_to_apy(apr / 100.0, compounds) * 100.0   # convert % -> fraction -> APY -> %
        for apr in aprs_percent
    ]
    return apys_percent

In [6]:
print("APRs in % for each pool  :" , compute_native_APR(1e18, 5*[100_000_000*1e18], 5*[500]))
print("APYs in % for same pools :", compute_native_APY(1e18, 5*[100_000_000*1e18], 5*[500], 365))

APRs in % for each pool  : [5.991840000000001, 5.991840000000001, 5.991840000000001, 5.991840000000001, 5.991840000000001]
APYs in % for same pools : [6.174468300782898, 6.174468300782898, 6.174468300782898, 6.174468300782898, 6.174468300782898]


In [ ]:
reward_rate = 1e18
num_tokens_per_pool = 5*[100_000_000*1e18]
fees_per_pool = [0] + 4*[500] # first pool owner takes no fee, other 4 owners take 5%
print("APRs in % for each pool  :" , compute_native_APR(reward_rate, num_tokens_per_pool, fees_per_pool))
print("APYs in % for same pools :", compute_native_APY(reward_rate, num_tokens_per_pool, fees_per_pool, 365))

APRs in % for each pool  : [6.307199999999999, 5.991840000000001, 5.991840000000001, 5.991840000000001, 5.991840000000001]
APYs in % for same pools : [6.509772041144912, 6.174468300782898, 6.174468300782898, 6.174468300782898, 6.174468300782898]


In [ ]:
reward_rate = 1e18
num_tokens_per_pool = [1_000_000*1e18] + 4*[100_000_000*1e18] # first OperatorStaking pool has -99% less tokens staked than the other 4 pools
fees_per_pool = 5*[500] # first pool owner takes no fee, other 4 owners take 5%
print("APRs in % for each pool  :" , compute_native_APR(reward_rate, num_tokens_per_pool, fees_per_pool))
print("APYs in % for same pools :", compute_native_APY(reward_rate, num_tokens_per_pool, fees_per_pool, 365))

APRs in % for each pool  : [73.07121951219511, 7.307121951219512, 7.307121951219512, 7.307121951219512, 7.307121951219512]
APYs in % for same pools : [107.5042726922077, 7.579928500245625, 7.579928500245625, 7.579928500245625, 7.579928500245625]
